 #  simple feed-forward neural network for MNIST

In [24]:
import tensorflow as tf
import math
from tensorflow.examples.tutorials.mnist import input_data

In [25]:
# dataset
data_sets = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


The fake_data flag is used for unit-testing purposes and may be safely ignored by the reader.

In [26]:
# placeholder
num_class = 10
images_placeholder = tf.placeholder(tf.float32, shape=(None,
                                                       28*28))
labels_placeholder = tf.placeholder(tf.int32, shape=(None,num_class))

In [27]:
# build graph
hidden_units = 100
weights = tf.Variable( tf.truncated_normal([28*28, hidden_units] , stddev = 1.0 / math.sqrt(float(28.28))))

biases = tf.Variable(tf.zeros([hidden_units]))

hidden1 = tf.nn.relu(tf.matmul(images_placeholder , weights) + biases)

weights_o = tf.Variable( tf.truncated_normal([hidden_units,num_class] , stddev = 1.0 / math.sqrt(float(28.28))))
biases_o = tf.Variable(tf.zeros([num_class]))

logits = tf.matmul(hidden1, weights_o) + biases_o

In [28]:
# loss
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=labels_placeholder, logits=logits))


In [29]:
# train and predict
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_placeholder, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [38]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(301):
        batch = data_sets.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
              images_placeholder: batch[0], labels_placeholder: batch[1]})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={images_placeholder: batch[0], labels_placeholder: batch[1]})
    
    batch_size = 50
    batch_num = int(data_sets.test.num_examples / batch_size)
    test_accuracy = 0

    for i in range(batch_num):
        batch = data_sets.test.next_batch(batch_size)
        test_accuracy += accuracy.eval(feed_dict={images_placeholder: batch[0],
                                              labels_placeholder: batch[1]})

    test_accuracy /= batch_num
    print("test accuracy %g"%test_accuracy)

step 0, training accuracy 0.12
step 100, training accuracy 0.2
step 200, training accuracy 0.46
step 300, training accuracy 0.62
test accuracy 0.5561
